In [ ]:
#https://anyscale.github.io/doc-prototype/tutorials/serving-a-pytorch-model
#https://docs.ray.io/en/latest/serve/tutorials/serve-ml-models.html

In [4]:
!pip install torch torchvision

     |███████████████████████████████▊| 882.9 MB 438 kB/s eta 0:00:170104     |██████████████████▌             | 514.6 MB 135.7 MB/s eta 0:00:03     |███████████████████████▎        | 647.2 MB 252 kB/s eta 0:16:02��██▍        | 651.0 MB 426 kB/s eta 0:09:21     |████████████████████████▋       | 684.0 MB 284 kB/s eta 0:12:05     |██████████████████████████████▏ | 839.7 MB 108.9 MB/s eta 0:00:01     |████████████████████████████████| 890.2 MB 1.8 kB/s 
     |████████████████████████████████| 24.3 MB 106 kB/s  eta 0:00:01
     |████████████████████████████████| 317.1 MB 6.0 kB/s eta 0:00:0101  |███████████████▏                | 150.0 MB 362 kB/s eta 0:07:42.4 MB 403 kB/s eta 0:06:02
     |████████████████████████████████| 557.1 MB 7.8 kB/s s eta 0:00:01                           | 18.9 MB 136 kB/s eta 1:05:45  | 25.0 MB 136 kB/s eta 1:05:00▌                             | 43.7 MB 221 kB/s eta 0:38:41     |████████████████▌               | 288.2 MB 230 kB/s eta 0:19:27MB 351 kB/s eta 0:07:

In [4]:
import ray
from ray import serve

from io import BytesIO
from PIL import Image
import requests
from anyscale import AnyscaleSDK

import torch
from torchvision import transforms
from torchvision.models import resnet18

runtime_env = {
 "pip": ["torch", "torchvision", "ray[serve]"]
}

/home/ray/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Hello World Serve example

In [5]:
if ray.is_initialized():
    print('Shutting down ray')
    ray.shutdown()
    
project_name = 'ray-demos/'
cluster_name = 'workspace-cluster-ray-demo-workspace'

ray.init(f"anyscale://{project_name}{cluster_name}", namespace="my_serve_namespace", runtime_env=runtime_env, autosuspend=-1)

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Parsing Ray Client arguments
[Warning] No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
Finished parsing arguments.

2022-12-12 18:28:04,299	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 10.0.63.8:9031...
2022-12-12 18:28:04,328	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_buwxbm99nq8dryqg6p8sbytw/services?redirect_to=dashboard 
2022-12-12 18:28:04,335	INFO packaging.py:373 -- Pushing file package 'gcs://_ray_pkg_7fe7c4f2c5b410a8a58ec6267d2342ce.zip' (0.13MiB) to Ray cluster...
2022-12

Python version:,3.9.12
Ray version:,2.2.0
Dashboard:,http://console.anyscale.com/api/v2/sessions/ses_buwxbm99nq8dryqg6p8sbytw/services?redirect_to=dashboard


In [6]:
serve.start(detached=True)

@serve.deployment
def hello(request):
    name = request.query_params["name"]
    return f"Hello {name}!"

hello.deploy()

(ServeController pid=35007) INFO 2022-12-12 18:28:09,047 controller 35007 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-44842751325a914ba902ee7d09b28631948f45f3ce9c61b23f516d93' on node '44842751325a914ba902ee7d09b28631948f45f3ce9c61b23f516d93' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=35049) INFO:     Started server process [35049]
(ServeController pid=35007) INFO 2022-12-12 18:28:10,688 controller 35007 deployment_state.py:1310 - Adding 1 replica to deployment 'hello'.


In [7]:
sdk = AnyscaleSDK()
cluster = sdk.search_clusters(clusters_query={"name": {"equals": cluster_name}}).results[0]

resp = requests.get(
   "http://localhost:8000"
   "/hello?name=Anyscale", headers={"Authorization" : f"Bearer {cluster.user_service_token}"})
print(resp.text)

Hello Anyscale!


In [8]:
cluster

{'access_token': 'aph0_CkgwRgIhAI_FF4sFQ_qXtxzSfrh25czJnpuUJl1IaFnwf-LZJ48hAiEAwyFEfzQC1G1z-iWidG2ddM2Nclggn9BxxOkeOyzQkS4SfxIgLYDi2p7ZnPHZX8Rfv8TIdiEs3vCX_Tua6wDN4mcOtRsYASIcdXNyXzRaV0hKUWRKamJ3ZkYydUJrWTF0NExYdjoMCNb59ucGELjWjb8DQgwIqvTenAYQuNaNvwP6AR4KHHNlc19idXd4Ym05OW5xOGRyeXFnNnA4c2J5dHc',
 'allow_public_internet_traffic': False,
 'cluster_compute_config': None,
 'cluster_compute_id': 'cpt_nhskcrdkxwldxas3i1svqmw5l8',
 'cluster_environment_build_id': 'anyscaleray220-py39',
 'created_at': datetime.datetime(2022, 12, 12, 23, 37, 58, 985482, tzinfo=tzutc()),
 'creator_id': 'usr_4ZWHJQdJjbwfF2uBkY1t4LXv',
 'goal_state': None,
 'ha_job_id': None,
 'head_node_info': {'ip_address': '54.70.107.192',
                    'url': 'https://session-buwxbm99nq8dryqg6p8sbytw.i.anyscaleuserdata.com'},
 'id': 'ses_buwxbm99nq8dryqg6p8sbytw',
 'idle_timeout_minutes': 120,
 'name': 'workspace-cluster-ray-demo-workspace',
 'project_id': 'prj_pifz5g7r6v1mzhnjm7l7wj12',
 'ray_version': '2.2.0',
 'ray_ve

In [9]:
serve.shutdown()

(HTTPProxyActor pid=35049) INFO 2022-12-12 18:28:12,853 http_proxy 10.0.63.8 http_proxy.py:361 - GET /hello 200 5.7ms
(ServeReplica:hello pid=35099) INFO 2022-12-12 18:28:12,851 hello hello#EyLCkv replica.py:505 - HANDLE __call__ OK 0.3ms
(ServeController pid=35007) INFO 2022-12-12 18:28:12,879 controller 35007 deployment_state.py:1336 - Removing 1 replica from deployment 'hello'.


# Image classification example with PyTorch model

In [10]:
@serve.deployment(route_prefix="/image_predict")
class ImageModel:
    def __init__(self):
        self.model = resnet18(pretrained=True).eval()
        self.preprocessor = transforms.Compose([
           transforms.Resize(224),
           transforms.CenterCrop(224),
           transforms.ToTensor(),
           transforms.Lambda(lambda t: t[:3, ...]),  # remove alpha channel
           transforms.Normalize(
               mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    async def __call__(self, starlette_request):
        image_payload_bytes = await starlette_request.body()
        pil_image = Image.open(BytesIO(image_payload_bytes))
        print("[1/3] Parsed image data: {}".format(pil_image))

        pil_images = [pil_image]  # Our current batch size is one
        input_tensor = torch.cat(
           [self.preprocessor(i).unsqueeze(0) for i in pil_images])
        print("[2/3] Images transformed, tensor shape {}".format(
           input_tensor.shape))

        with torch.no_grad():
            output_tensor = self.model(input_tensor)
        print("[3/3] Inference done!")
        return {"class_index": int(torch.argmax(output_tensor[0]))}

In [11]:
#ray.init("anyscale://workspace-cluster-kyle-sandbox", namespace="my_serve_namespace", runtime_env=runtime_env, autosuspend=-1)

serve.start(detached=True)

ImageModel.deploy()

(ServeController pid=35168) INFO 2022-12-12 18:28:17,864 controller 35168 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-44842751325a914ba902ee7d09b28631948f45f3ce9c61b23f516d93' on node '44842751325a914ba902ee7d09b28631948f45f3ce9c61b23f516d93' listening on '127.0.0.1:8000'
(ServeController pid=35168) INFO 2022-12-12 18:28:19,840 controller 35168 deployment_state.py:1310 - Adding 1 replica to deployment 'ImageModel'.
(HTTPProxyActor pid=35214) INFO:     Started server process [35214]


In [12]:
from anyscale import AnyscaleSDK

sdk = AnyscaleSDK()
cluster = sdk.search_clusters(clusters_query={"name": {"equals": cluster_name}}).results[0]

ray_logo_bytes = requests.get(
   "https://github.com/ray-project/ray/raw/"
   "master/doc/source/images/ray_header_logo.png").content

resp = requests.post(
   f"{cluster.services_urls.user_service_url}/image_predict", headers={"Authorization" : f"Bearer {cluster.user_service_token}"}, data=ray_logo_bytes)
   #f"http://localhost:8000/image_predict", headers={"Authorization" : f"Bearer {cluster.user_service_token}"}, data=ray_logo_bytes)
print(resp.json())

(ServeReplica:ImageModel pid=35261) [1/3] Parsed image data: <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=4726x1478 at 0x7FE89C1D82B0>
{'class_index': 919}


In [13]:
serve.shutdown()

(HTTPProxyActor pid=35214) INFO 2022-12-12 18:28:24,595 http_proxy 10.0.63.8 http_proxy.py:361 - POST /image_predict 200 309.6ms
(ServeReplica:ImageModel pid=35261) INFO 2022-12-12 18:28:24,593 ImageModel ImageModel#srjLUF replica.py:505 - HANDLE __call__ OK 303.7ms


(ServeReplica:ImageModel pid=35261) [2/3] Images transformed, tensor shape torch.Size([1, 3, 224, 224])
(ServeReplica:ImageModel pid=35261) [3/3] Inference done!


(ServeController pid=35168) INFO 2022-12-12 18:28:24,612 controller 35168 deployment_state.py:1336 - Removing 1 replica from deployment 'ImageModel'.


# Serve from CLI

In [ ]:
# Run following CLI in term
#!serve run pytorch_serve:image_model

In [14]:
import requests
ray_logo_bytes = requests.get(
    "https://raw.githubusercontent.com/ray-project/"
    "ray/master/doc/source/images/ray_header_logo.png"
).content

resp = requests.post("http://localhost:8000/", data=ray_logo_bytes)
print(resp.json())

{'class_index': 919}
(scheduler +16m9s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +16m9s) Adding 1 node(s) of type worker-node-type-0.
(scheduler +18m20s) Resized to 24 CPUs.
(scheduler +23m23s) Removing 1 nodes of type worker-node-type-0 (idle).
(scheduler +23m34s) Resized to 8 CPUs.
